# Google API Challenge with Python 



1. Setup a test gmail account and enable the APIs for it.
2. Send e-mails.
3. Search for specific messages in the mailbox.



**Prerequisites**
1. Python 2.6 or greater.
2. The pip package management tool.


## 1- Setup a test gmail account and enable the API and other 

We first create a new gmail account: In our case is haensel.challenge@gmail.com and the password is passwordChallengeAPI



### 1.1 Create a Google Cloud project 

At the top-left, click Menu > IAM & Admin > Create a Project

Open the [Google Cloud console](
https://console.cloud.google.com/)

<img src="create_project_cloud.png" width="400"/>

Add the Project name and Location.


### 1.2 OAuth consent screen  


At the top-left, click Menu > APIs & Services > OAuth consent screen 

Set User Type as External.

<img src="Oauth1_cloud.png" width="400"/>

Set the App information, App domain and the developer e-mail, click Save and Continue.

<img src="Oauth_cloud.png" width="400"/>



Skip Add Scopes, since we could add them manually in our script, click Save and Continue.

<img src="scope_cloud.png" width="400"/>


In the Test users section click Add users. Enter your e-mail address and any other authorized test users, click Save and Continue.

<img src="testusers_cloud.png" width="400"/>


### 1.3  Enable the GMAIL API

At the top-left, click Menu > APIs & Services > Library

<img src="library_cloud.png" width="400"/>

Search for GMAIL API and click on ENABLE.

<img src="enable_cloud.png" width="400"/>

### 1.4 Create credentials 

At the top-left, click Menu > APIs & Services > Credentials  > + CREATE CREDENTIALS  > Create OAuth client ID

Set the application type as Desktop.

Set the name of your OAuth 2.0 client.

Download the credentials: 

<img src="get_credentials_cloud.png" width="500"/>


<img src="download_cloud.png" width="400"/>

**IMPORTANT STEP**: You would get a .json file, rename it as credentials.json and save it your working directory.

### 1.5 Install the Google API library


In [1]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


### 1.6 Authentification  



We first import the necessary libraries, we follow parts of the quickstart guide to make authorization easier (https://github.com/googleworkspace/python-samples/blob/main/gmail/quickstart/quickstart.py)

We import [pickle](https://pypi.org/project/pickle5/) to store the authentication credentials.

We also import [mimetypes](https://pypi.org/project/mime/). A MIME is an identifier for a particular type of format of information (Read more here:https://docs.python.org/3/library/mimetypes.html) 

Select one of the Scopes (Scopes are different levels of permission that an app could request from an user).
https://developers.google.com/gmail/api/auth/scopes






In [2]:
pip install pickle5

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mime

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import pickle
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from email.mime.image import MIMEImage
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.audio import MIMEAudio
from email.mime.base import MIMEBase
from apiclient import errors
import mimetypes
import base64
import email


# If modifying these scopes, delete the file token.json.
SCOPES=['https://mail.google.com/']

In [5]:
def authentication():
    """
    """
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)
    return build('gmail', 'v1', credentials=creds)

# get the Gmail API service

service = authentication()

### 1.7 Authorize the application 

Choose your account and press continue 
<img src="authorization_1.png" width="400"/>


## 2- Send e-mails from GMAIL API
First note that the single MIME message part has the following JSON representation:




### 2.1 Create a message with attachment 
We follow the guidelines of https://developers.google.com/gmail/api/guides/sending

**STEPS**
We are using two functions:
1. Create message with attachment 
2. Send messages 



In [6]:
def create_message_with_attachment(sender,to,subject,message_text,file):
    """
    Creates a message with attachement for an e-mail
    
    
    sender: string with the e-mail address of the sender.
    to: string with the e-mail address of the receiver.
    subject: subject of the e-mail message.
    message_text: the text of the e-mail message.
    
    
    Returns the full e-mail message data with body content in the raw field as a base64url encoded string
    """
    
    #headers
    message = MIMEMultipart()
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject

    msg = MIMEText(message_text)
    message.attach(msg)

    #guessing the MIME type of the file
    (content_type, encoding) = mimetypes.guess_type(file)
    
    #creates a MIME type for the file
    if content_type is None or encoding is not None:
        content_type = 'application/octet-stream'

    (main_type, sub_type) = content_type.split('/', 1)

    if main_type == 'text':
        with open(file, 'rb') as f:
            msg = MIMEText(f.read().decode('utf-8'), _subtype=sub_type)

    elif main_type == 'image':
        with open(file, 'rb') as f:
            msg = MIMEImage(f.read(), _subtype=sub_type)
        
    elif main_type == 'audio':
        with open(file, 'rb') as f:
            msg = MIMEAudio(f.read(), _subtype=sub_type)
        
    else:
        with open(file, 'rb') as f:
            msg = MIMEBase(main_type, sub_type)
            msg.set_payload(f.read())

    filename = os.path.basename(file)
    msg.add_header('Content-Disposition', 'attachment',
                   filename=filename)
   
    #Give the message as a base64-encoded string.

    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode()
        
    return {'raw': raw_message}

### 2.2 Send the message 


In [7]:
def send_message(service, user_id, message):
    """Send an e-mail message
    
    user_id: string
    message: A base64-encoded string.

    Prints: the returned message id
    Returns: message object and id 

    """
    try:
        message = service.users().messages().send(userId=user_id,
                body=message).execute()

        print('Message Id: {}'.format(message['id']))

        return message
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

### 2.3 Example 

You could also change below the recipient, the subject, the body and the attached file by changing to, subject, body and file respectively. 

In [8]:
if __name__ == '__main__':
    
    #Load pre-authorized user credentials from the environment.
    service=authentication()
    
    user_id='me'
    sender='haensel.challenge@gmail.com'
    to='alrinconh@gmail.com'
    subject='How to pass the challenge?'
    body='steb by step'
    file='mensaje.txt'
    msg=create_message_with_attachment(sender,to,subject, body, file)
    send_message(service,user_id,msg)


Message Id: 1831860a25e08c46


## 3-Search specific messages 

Task: Search for e-mails that satisfy a search_string given as in https://support.google.com/mail/answer/7190

We follow https://developers.google.com/gmail/api/guides/filtering and https://developers.google.com/gmail/api/reference/rest/v1/users.messages/get

We define a function to search and display part of the messages. We first get the id of the messages satisfying the search_string and then we obtain their sender, subject and a snippet from the content. 

To see some example of the outcome of service.users().messages().list(userId=user_id, q=search_string).execute() and 
service.users().messages().get(userId=user_id, id=msg_id).execute() would help us to understand our 
code. 

### Example 1

For userID='me'

search_string='message'
   
search_output=service.users().messages().list(userId=user_id, q=search_string).execute()

search_output={'messages': [{'id': '182f8708b31304d3', 'threadId': '182f8708b31304d3'}],'resultSizeEstimate': 1}

### Example 2

Run the code below:

In [9]:
message = service.users().messages().get(userId='me', id='182eef80ee4cb393').execute()
msg_data=message['payload']['headers']
msg_data 


[{'name': 'Delivered-To', 'value': 'haensel.challenge@gmail.com'},
 {'name': 'Received',
  'value': 'by 2002:a59:a381:0:b0:2de:ed6b:aeae with SMTP id l1csp2457883vqo;        Tue, 30 Aug 2022 06:36:54 -0700 (PDT)'},
 {'name': 'X-Received',
  'value': 'by 2002:a0c:8d85:0:b0:497:8b1:d372 with SMTP id t5-20020a0c8d85000000b0049708b1d372mr15215612qvb.68.1661866611367;        Tue, 30 Aug 2022 06:36:51 -0700 (PDT)'},
 {'name': 'ARC-Seal',
  'value': 'i=1; a=rsa-sha256; t=1661866610; cv=none;        d=google.com; s=arc-20160816;        b=HnQ+MXMs8aS1WaziIC42RE/K4J5Yms8nq114ikADwEf+6RHp8Wh6BqNL65xhpmnMGd         7jkX4Btlb4WYAgemMZ75jBWOyUNIJFHMRbA814CyZFZnjP9T3u9mXtJtsSV+j20r53mQ         BWOiDwRLwwQhyVe7DHYF4c6f5uZkIa+g5ZKvZgugRNXes/GtgbSctAgnzT2Gez9yP7e+         ZFIfNpo5zbr0RjlLEPy5RIjemeXiRCZXqLuhkGnWn/2IIaUuDcxaJEssf2LRXsRkY+k8         tvfN2g90KFG99Kj8j2JHXkqta0GR6mBNwhIE1MOsW3gKVFDObm/yaYyW5TEXGGnvszwT         oOTA=='},
 {'name': 'ARC-Message-Signature',
  'value': 'i=1; a=rsa-sha256; c=rel

In [10]:
def search_message(service, user_id, search_string):
    """
    Search for e-mails that satisfy the search_string
    
    
        user_id: a string ('me' works here if
        we already authorized it)
        search_string: 
    Prints: Sender, Subject and a snippet from the e-mails that satisfy the search_string
    Returns: Sender, Subject and a snippet from the e-mails that satisfy the search_string
    """
    try:
        ids_list = []

        # Get the ids of the messages that contain the search_string
        search_output = service.users().messages().list(userId=user_id, q=search_string).execute()
        # If there are no results e.g. search_output={'resultSizeEstimate': 0}, then print a warning and return empty string
        try:
            Ids = search_output['messages']

        except KeyError:
            print("WARNING: 0 results")
            print("it returns an empty string")
            return ""
        
        
        for msg_id in Ids:
                ids_list.append(msg_id['id'])
        
        # Print the ids of the messages that contain the search_string
        print(ids_list)  
    except (errors.HttpError, error):
        print("An error occured: %s") % error

    # Find the sender, subject and a snippet of the messages satisfying the search string 
    try:
        for msg_id in ids_list:
            message = service.users().messages().get(userId=user_id, id=msg_id).execute()
           
            msg_data=message['payload']['headers']
            
            for val in msg_data:
                name=val["name"]
                if name=='From':
                    from_email=val['value']
                    print('From: '+ from_email)
                if name=='Subject':
                    subject_email=val['value']
                    print('Subject: '+ subject_email)
            print(message['snippet'])
            print('\n')
            
        return(from_email, subject_email, message['snippet'])
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None


### 3.1 Example

You could also change the search_string below to obtain new outcomes. 

In [11]:
service=authentication()
    
user_id='me'
search_string='search'
msgs_searched=search_message(service, user_id, search_string)
    


['182f8708b31304d3', '182f85bcc49dcbc2', '182eef80ee4cb393']
From: Alejandra Rincon Hidalgo <alrinconh@gmail.com>
Subject: Another message to search
Seconde message


From: Alejandra Rincon Hidalgo <alrinconh@gmail.com>
Subject: Message to be search
This is the message that I am using to search for it. We add additional words to look for it: API Google Python Data Scientist


Subject: Your Google Account is live – now help your business grow
From: Google Community Team <googlecommunityteam-noreply@google.com>
Start using Google&#39;s tools for your business. ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍ ‍


